# [Lv4-Day3-Lab1] Building a ReAct Agent

### 실습 목표
3일차의 첫 실습에서는 단일 Tool의 한계를 넘어, 여러 개의 이질적인 Tool들을 상황에 맞게 **동적으로 선택하고 조합**하여 복잡한 문제를 해결하는 ** ReAct Agent**를 밑바닥부터(from scratch) 구현합니다. 이 Agent는 단순한 어시스턴트가 아닌, **문제 해결사**로서 두 가지 상이한 시나리오—**수능 문제 풀이**와 **블로그 초안 생성**—를 모두 처리할 수 있는 유연성을 갖추게 됩니다.

1.  **확장 가능한 Tool 시스템 설계:** 웹 검색, 간단한 계산, 복잡한 수식 풀이(WolframAlpha), 사용자 상호작용, 계획 수립 등 6가지 전문 Tool을 모듈식으로 구현합니다.
2.  **구조화된 행동(Structured Action):** 1일차에 배운 `Pydantic`과 `Instructor`를 ReAct 패턴에 통합합니다. LLM이 생성하는 'Action'을 불안정한 텍스트가 아닌, **100% 신뢰할 수 있는 Pydantic 객체**로 강제하여 Agent의 안정성을 극대화합니다.
3.  **지능형 Tool 라우팅:** Agent의 `Thought` 단계에서, LLM이 문제의 본질을 파악하여 **어떤 Tool이 현재 상황에 가장 적합한지 스스로 추론하고 선택**하는 핵심 로직을 이해합니다.
4.  **자율적 루프 제어:** Agent가 **스스로 문제 해결의 완료를 판단**하고 `Final Answer`를 통해 작업을 종료하는 자율적인 제어 흐름을 구현합니다.

### 0. 사전 준비: 라이브러리 설치 및 API 키 설정
이번 실습에서는 복잡한 수식 계산을 위해 **WolframAlpha LLM API**를 새롭게 사용합니다.

In [1]:
# !pip install langchain-google-genai langchain-community tavily-python streamlit wolframalpha pydantic instructor jsonref request -q

In [ ]:
import os
from getpass import getpass

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = "your-api-key"

if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = "your-api-key"

# WolframAlpha App ID를 발급받아 환경변수에 저장해야 합니다.
# LLM API Document: https://products.wolframalpha.com/llm-api/documentation
# App ID 발급: https://developer.wolframalpha.com/portal/myapps/
if "WOLFRAM_ALPHA_APP_ID" not in os.environ:
    os.environ["WOLFRAM_ALPHA_APP_ID"] = "your-api-key"

print("✅ 모든 API 키 및 토큰이 성공적으로 설정되었습니다.")

✅ 모든 API 키 및 토큰이 성공적으로 설정되었습니다.


### 1. Agent의 능력: 다중 Tool 시스템 구축 (The Toolbox)
우리 Agent가 다양한 문제를 해결할 수 있도록, 각기 다른 전문성을 가진 6개의 Tool로 구성된 'Toolbox'를 구축합니다. 사용자님께서 완성해주신 이전 실습의 모든 Tool 기능을 포함하고, 각 Tool의 `description`을 LLM이 명확히 이해하고 구분할 수 있도록 상세하게 작성합니다.

In [4]:
from langchain.tools import tool, BaseTool
from langchain_community.tools.tavily_search import TavilySearchResults
from pydantic import Field, BaseModel, PrivateAttr
from typing import Type, List
import urllib.request
import urllib.parse

# --- Tool 1: 웹 검색 ---
search_tool = TavilySearchResults(max_results=3, name="tavily_search_results_json")


# --- Tool 2: 간단한 계산기 ---
class CalculatorInput(BaseModel):
    expression: str = Field(description="평가할 수학적 표현식")


@tool(args_schema=CalculatorInput)
def calculator_tool(expression: str) -> str:
    """간단한 사칙연산이나 숫자 계산에만 사용하세요."""
    try:
        return str(eval(expression))
    except Exception as e:
        return f"계산 오류: {e}"


# --- Tool 3: 복합 과학/수학 엔진 (WolframAlpha LLM API) ---
class WolframAlphaInput(BaseModel):
    query: str = Field(description="WolframAlpha에 보낼 복잡한 질문이나 수학/과학 수식")


@tool(args_schema=WolframAlphaInput)
def wolfram_alpha_tool(query: str) -> str:
    """복잡한 수학 문제, 방정식, 미적분, 화학식, 물리 공식, 단위 변환 등에 사용합니다."""
    try:
        import urllib.request
        import urllib.parse

        api_key = os.environ.get("WOLFRAM_ALPHA_APP_ID")
        if not api_key:
            return "API 키가 설정되지 않았습니다."

        # LLM API 사용 (텍스트 결과 반환)
        base_url = "https://www.wolframalpha.com/api/v1/llm-api"

        # URL 파라미터 구성
        params = {"input": query, "appid": api_key, "maxchars": 2000}  # 응답 길이 제한

        # URL 인코딩
        encoded_params = urllib.parse.urlencode(params)
        full_url = f"{base_url}?{encoded_params}"

        print(f"디버깅: 요청 URL: {full_url}")

        # 요청 실행
        with urllib.request.urlopen(full_url) as response:
            result = response.read().decode("utf-8")
            print(f"디버깅: LLM API 응답 성공")

            # 결과에서 핵심 정보만 추출
            if "Result:" in result:
                # "Result:" 다음 부분만 추출
                result_section = result.split("Result:")[1].split("\n")[0:3]
                clean_result = "Result: " + "\n".join(result_section)
                return clean_result.strip()

            return result[:500] + "..." if len(result) > 500 else result

    except urllib.error.HTTPError as e:
        error_code = e.code
        error_msg = e.read().decode("utf-8") if hasattr(e, "read") else str(e)

        if error_code == 501:
            return f"WolframAlpha가 쿼리를 이해하지 못했습니다: '{query}'. 다른 표현으로 시도해보세요."
        elif error_code == 403:
            return "API 키가 유효하지 않거나 권한이 없습니다."
        else:
            return f"HTTP 오류 {error_code}: {error_msg}"

    except Exception as e:
        return f"WolframAlpha 오류: {str(e)}"


# --- Tool 4: 블로그 템플릿 ---
class BlogTemplateInput(BaseModel):
    style: str = Field(
        description="블로그 글의 기본 구조(템플릿)를 생성할 때 사용합니다. '기술 분석' 또는 '제품 리뷰' 스타일 중 하나를 선택하세요."
    )


@tool(args_schema=BlogTemplateInput)
def blog_template_tool(style: str) -> str:
    """블로그 글의 기본 구조를 생성합니다."""
    if style == "기술 분석":
        return "## 제목\\n\\n### 1. 기술 개요\\n\\n### 2. 핵심 작동 원리\\n\\n### 3. 장단점 분석\\n\\n### 4. 실무 적용 사례\\n\\n### 5. 결론 및 향후 전망"
    elif style == "제품 리뷰":
        return "## 제목\\n\\n### 1. 첫인상 및 디자인\\n\\n### 2. 주요 기능 및 성능 테스트\\n\\n### 3. 실사용 후기\\n\\n### 4. 총평 및 추천 대상"
    else:
        return "오류: '기술 분석' 또는 '제품 리뷰' 스타일만 지원합니다."


# --- Tool 5: 사용자에게 질문하기 ---
class QuestionInput(BaseModel):
    question: str = Field(description="사용자에게 물어볼 질문")


@tool(args_schema=QuestionInput)
def ask_user_tool(question: str) -> str:
    """사용자에게 추가 정보를 질문할 때 사용합니다. 계획 수립이나 맞춤 조언을 위해 필요한 정보를 얻습니다."""
    return f"사용자에게 질문: {question}"


# --- Tool 6: 맞춤형 계획 생성 ---
class PlanningInput(BaseModel):
    user_info: str = Field(description="사용자 정보나 상황")
    goal: str = Field(description="달성하고자 하는 목표")


@tool(args_schema=PlanningInput)
def create_study_plan_tool(user_info: str, goal: str) -> str:
    """사용자 정보를 바탕으로 맞춤형 학습 계획을 생성합니다."""

    # 간단한 계획 생성 로직
    if "수능" in goal.lower() or "수학" in goal.lower():
        return f"""
## 📚 맞춤형 수능 수학 학습 계획

**사용자 현황:** {user_info}
**목표:** {goal}

### 1단계: 현재 실력 점검 (1주차)
- 기출문제 3개년 풀어보기
- 약점 영역 파악

### 2단계: 개념 정리 (2-4주차)
- 부족한 단원 집중 학습
- 공식 암기 및 이해

### 3단계: 문제 유형별 연습 (5-8주차)
- 킬러 문제 유형 분석
- 시간 단축 연습

### 4단계: 실전 모의고사 (9-12주차)
- 주 2회 모의고사
- 오답 노트 작성
        """
    elif "블로그" in goal.lower():
        return f"""
## ✍️ 맞춤형 블로그 시작 계획

**사용자 현황:** {user_info}
**목표:** {goal}

### 1단계: 주제 및 타겟 독자 설정
- 관심 분야와 전문성 연결
- 독자층 명확화

### 2단계: 콘텐츠 전략 수립
- 포스팅 주기 결정
- 콘텐츠 캘린더 작성

### 3단계: 첫 포스팅 발행
- 자기소개 글 작성
- SEO 최적화 적용
        """


# 최종 Tool 리스트
tools = [search_tool, calculator_tool, wolfram_alpha_tool, blog_template_tool, ask_user_tool, create_study_plan_tool]
print(f"✅ 총 {len(tools)}개의 Tool이 성공적으로 준비되었습니다.")

✅ 총 6개의 Tool이 성공적으로 준비되었습니다.


/tmp/ipykernel_1098/2799469966.py:9: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search_tool = TavilySearchResults(max_results=3, name="tavily_search_results_json")


### 2. ReAct의 '행동' 명세서: Pydantic Action 모델 정의
이제 ReAct 패턴의 안정성을 극대화하기 위한 핵심 단계입니다. LLM이 생성할 `Action`을 불안정한 텍스트가 아닌, **엄격하게 검증된 Pydantic 객체**로 정의합니다. 이를 '행동의 설계도(Blueprint for Action)'라고 할 수 있습니다.

`tool` 필드는 `Literal` 타입을 사용하여 우리가 정의한 Tool의 이름 또는 'Final Answer' 외에는 절대 다른 값이 올 수 없도록 강제합니다. `tool_input`은 각 Tool이 요구하는 다양한 입력 형식(문자열 또는 딕셔너리)을 모두 받을 수 있도록 `Union`을 사용합니다.

In [5]:
from pydantic import BaseModel, Field
from typing import Literal, Any


class Action(BaseModel):
    """LLM이 따라야 할 행동의 스키마입니다."""

    thought: str = Field(description="현재 상황 분석과 다음 행동 계획")

    tool: Literal[
        "tavily_search_results_json",
        "calculator_tool",
        "wolfram_alpha_tool",
        "blog_template_tool",
        "ask_user_tool",
        "create_study_plan_tool",
        "Final Answer",
    ] = Field(description="사용할 Tool의 이름 또는 최종 답변을 위한 'Final Answer'")

    tool_input: Any = Field(description="선택된 Tool에 전달할 입력값. 문자열 또는 JSON 객체 형태가 될 수 있습니다.")


print("✅ ReAct Action을 위한 Pydantic 모델이 성공적으로 정의되었습니다.")

✅ ReAct Action을 위한 Pydantic 모델이 성공적으로 정의되었습니다.


### 3. ReAct의 '두뇌': 프롬프트와 구조화된 엔진 구현

이제 이 다양한 Tool들을 지능적으로 사용할 ReAct의 핵심 엔진을 만듭니다. 여기서 가장 중요한 두 가지는 다음과 같습니다.

1.  **지능형 프롬프트 (Intelligent Prompt):** LLM에게 '문제 해결사'라는 페르소나를 부여하고, 우리가 만든 Tool들의 명세서를 제공하며, **ReAct 패턴을 따르되 Action은 반드시 우리가 정의한 Pydantic 모델의 JSON 스키마를 준수하도록** 강력하게 지시합니다.

2.  **견고한 엔진 (Robust Engine):** 1일차에 배운 `instructor`를 사용하여 LLM의 출력을 우리가 정의한 `Action` Pydantic 객체로 직접 받습니다. 이를 통해 **불안정한 정규표현식(regex) 파싱을 완전히 제거**하여, Agent의 모든 행동이 100% 예측 가능하고 안정적으로 실행됨을 보장합니다.

In [6]:
from langchain.tools.render import render_text_description
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
import instructor
import json

# 1. LLM 초기화
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", temperature=0)

# 2. Instructor 클라이언트 생성: LLM이 Pydantic 객체를 출력하도록 '패치'합니다.
structured_llm = instructor.from_provider("google/gemini-2.5-flash-lite")


# 3. ReAct를 위한 프롬프트 템플릿 생성 함수
def get_react_prompt_template():
    """ReAct 패턴과 Pydantic 스키마를 포함한 시스템 프롬프트를 생성합니다."""
    # Tool들의 명세서를 텍스트로 렌더링
    rendered_tools = render_text_description(tools)

    # Pydantic 모델로부터 JSON 스키마 생성 (중괄호 이스케이프)
    action_schema = json.dumps(Action.model_json_schema(), indent=2).replace("{", "{{").replace("}", "}}")

    template = f"""
    # MISSION
    당신은 사용자의 복잡한 요구사항을 해결하는 최상위 AI 어시스턴트입니다.
    당신은 'Thought'와 'Action'을 반복하는 ReAct 패턴을 사용하여 문제를 단계적으로 해결해야 합니다.

    # TOOLS
    다음은 당신이 사용할 수 있는 도구 목록입니다:
    --- TOOLS ---
    {rendered_tools}
    --- END TOOLS ---

    # OUTPUT FORMAT
    당신의 응답은 반드시 다음 세 가지 키를 포함한 JSON 객체여야 합니다:
    1. 'thought': 현재 상황 분석과 다음 행동 계획을 서술하는 문자열
    2. 'tool': 사용할 도구의 이름 또는 'Final Answer'
    3. 'tool_input': 선택된 도구에 전달할 입력값

    --- ACTION SCHEMA ---
    {action_schema}
    --- END ACTION SCHEMA ---

    # WORKFLOW
    1. 사용자의 질문과 이전 대화 기록을 분석하여 현재 상황을 파악합니다.
    2. 'thought'에 다음 행동 계획을 상세히 서술합니다.
    3. 계획에 가장 적합한 도구와 입력값을 결정합니다.
    4. 만약 모든 정보가 수집되어 최종 답변을 할 수 있다면, 'tool'로 'Final Answer'를 사용합니다.

    --- CONTEXT ---
    이전 대화 기록:
    {{chat_history}}

    이전 행동 및 관찰 기록:
    {{intermediate_steps}}

    사용자 질문: {{user_query}}
    """
    return template


print("✅ ReAct 프롬프트 템플릿이 성공적으로 정의되었습니다.")

✅ ReAct 프롬프트 템플릿이 성공적으로 정의되었습니다.


#### 4. ReAct 엔진 함수 구현

이제 `instructor`를 사용하여 ReAct 루프의 핵심 로직을 구현합니다. 이전 실습의 불안정한 `re.search`와 `json.loads`가 `structured_llm.invoke(..., response_model=Action)` 이라는 단 한 줄의 **안정적이고 선언적인 코드**로 대체되는 것에 주목하세요.

In [7]:
from langchain_core.runnables import RunnablePassthrough


def run_structured_react_engine(user_query: str, chat_history: list):
    prompt_template = ChatPromptTemplate.from_template(get_react_prompt_template())
    tool_map = {tool.name: tool for tool in tools}

    history_str = "\n".join([f"Human: {h[0]}\nAssistant: {h[1]}" for h in chat_history])
    intermediate_steps_str = ""

    # 자율적 루프 시작
    for i in range(10):  # 최대 10번의 반복으로 안전장치 설정
        print(f"\n--- 🔄 ReAct Loop: Iteration {i+1} ---")

        # 1. 프롬프트 생성
        prompt = prompt_template.format(
            user_query=user_query, chat_history=history_str, intermediate_steps=intermediate_steps_str
        )

        # 2. instructor를 사용하여 구조화된 응답 생성
        response_object = structured_llm.chat.completions.create(
            model="gemini-2.5-flash", response_model=Action, messages=[{"role": "user", "content": prompt}]
        )

        thought = response_object.thought
        action_obj = response_object.action if hasattr(response_object, "action") else response_object

        print(f"🤔 Thought: {thought}")
        print(f"🎬 Action: {action_obj.tool}({action_obj.tool_input})")

        # 3. 최종 답변인지 확인
        if action_obj.tool == "Final Answer":
            print("\n✅ 최종 답변 결정")
            return action_obj.tool_input, chat_history

        # 4. Tool 실행
        if action_obj.tool in tool_map:
            tool_to_use = tool_map[action_obj.tool]
            tool_input = action_obj.tool_input
            observation = tool_to_use.invoke(tool_input)
            print(f"👀 관찰 결과 (일부): {str(observation)[:2000]}...")
        else:
            observation = f"오류: '{action_obj.tool}' Tool을 찾을 수 없습니다."

        # 5. 다음 루프를 위한 기록 업데이트
        intermediate_steps_str += (
            f"\nThought: {thought}\nAction: {action_obj.model_dump_json()}\nObservation: {observation}"
        )

    return "최대 반복 횟수에 도달하여 작업을 종료합니다.", chat_history


print("✅ Structured ReAct 엔진 'run_structured_react_engine'이 성공적으로 정의되었습니다.")

✅ Structured ReAct 엔진 'run_structured_react_engine'이 성공적으로 정의되었습니다.


### 5. ReAct 엔진 시뮬레이션 테스트
Streamlit UI를 만들기 전에, 우리의 새로운 '구조화된' ReAct 엔진이 복잡한 수능 문제를 어떻게 단계적으로 해결하는지 시뮬레이션을 통해 관찰합니다. Agent가 `Thought`를 통해 문제를 분해하고, 각 단계에 맞는 Tool(`wolfram_alpha`, `calculator_tool`)을 정확히 선택하여 사용하는지 주목하세요.

In [8]:
# 시뮬레이션 시작
chat_history_sim = []

print("--- 🚀 Structured ReAct 시뮬레이션 시작 ---")

query = "함수 f(x) = 2x^3 - 9x^2 + 12x - 2 의 극대값을 찾아줘. 그 다음, 그 극대값에 10을 더한 결과를 알려줘."

final_answer, chat_history_sim = run_structured_react_engine(query, chat_history_sim)

print("\n" + "=" * 50)
print("🤖 최종 응답:")
print(final_answer)

--- 🚀 Structured ReAct 시뮬레이션 시작 ---

--- 🔄 ReAct Loop: Iteration 1 ---
🤔 Thought: 주어진 함수 f(x) = 2x^3 - 9x^2 + 12x - 2의 극대값을 찾기 위해 wolfram_alpha_tool을 사용합니다. 극대값을 찾은 후에는 그 값에 10을 더해야 합니다. wolfram alpha에서 극대값을 구하는 쿼리를 작성합니다.
🎬 Action: wolfram_alpha_tool(local maximum of f(x) = 2x^3 - 9x^2 + 12x - 2)
디버깅: 요청 URL: https://www.wolframalpha.com/api/v1/llm-api?input=local+maximum+of+f%28x%29+%3D+2x%5E3+-+9x%5E2+%2B+12x+-+2&appid=73Q6KAVRGG&maxchars=2000
디버깅: LLM API 응답 성공
👀 관찰 결과 (일부): Result: 
max{2 x^3 - 9 x^2 + 12 x - 2} = 3 at x = 1...

--- 🔄 ReAct Loop: Iteration 2 ---
🤔 Thought: 이전 관찰 결과에서 함수 f(x) = 2x^3 - 9x^2 + 12x - 2의 극대값이 x=1에서 3이라는 것을 확인했습니다.
이제 이 극대값 3에 10을 더해야 합니다. 이는 간단한 사칙연산이므로 calculator_tool을 사용하여 계산합니다.
계산 결과가 나오면 최종 답변을 제공합니다.
🎬 Action: calculator_tool(3 + 10)
👀 관찰 결과 (일부): 13...

--- 🔄 ReAct Loop: Iteration 3 ---
🤔 Thought: 이전 대화 기록을 통해 사용자의 질문에 대한 극대값 계산과 10을 더하는 과정이 이미 완료되었음을 확인했습니다.
최종 결과는 13이며, 이제 이 결과를 사용자에게 최종 답변으로 제시하면 됩니다.
🎬 Action: Final Answer(함수 f(x) = 2x^3 - 9


**추천 테스트 시나리오:**
1.  **수능 문제 풀이:** `x^3 - 3x^2 + 5x + 1 = 0`의 실근을 찾아줘. (WolframAlphaTool 호출 유도)
2.  **블로그 초안 생성:** `LangChain의 LCEL에 대한 기술 분석 블로그`를 쓰고 싶어. 어떤 구조가 좋을까? (BlogTemplateTool 및 TavilySearchTool 호출 유도)
3.  **계획 수립:** `수능 수학 공부 계획을 세우고 싶어.` -> 이후 추가 질문에 대한 답변 -> `현재 3등급, 목표 1등급, 하루 4시간 가능, 확률과 통계 전체적으로 어려워`

**관찰 포인트:**
- **(Notebook 출력창)** Agent가 각 단계마다 어떤 `Thought`를 하고, 어떤 `Action`을 선택하는지 실시간 로그를 관찰하세요
- **(Streamlit UI)** 최종 결과물이 나오기까지 Agent가 어떻게 사용자와 상호작용(필요시)하거나 내부적으로 작업을 수행하는지 확인하세요